# Market Analysis: Sentiment vs. Base Models
This interactive dashboard compares the performance of Random Forest and LSTM models, with and without sentiment features, against a Buy & Hold strategy.

## Controls
- **Range Slider**: Use the slider at the bottom to zoom in on specific time periods.
- **Legend**: Click on legend items to toggle visibility. Double-click to isolate a trace.
- **Hover**: Hover over the chart to see precise values for each day.

In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import glob

# Configuration
DATA_DIR = 'results' 

# Find all tickers based on file pattern 'test_data_{ticker}.csv'
files = glob.glob(os.path.join(DATA_DIR, 'test_data_*.csv'))
tickers = [f.split('test_data_')[1].replace('.csv', '') for f in files]
tickers.sort()

print(f"Found tickers: {tickers}")

def load_ticker_data(ticker):
    test_path = os.path.join(DATA_DIR, f'test_data_{ticker}.csv')
    equity_path = os.path.join(DATA_DIR, f'equity_curves_{ticker}.csv')
    
    try:
        df_prices = pd.read_csv(test_path, index_col=0, parse_dates=True)
        df_equity = pd.read_csv(equity_path, index_col=0, parse_dates=True)
        return df_prices, df_equity
    except FileNotFoundError:
        print(f"Error: Data files for {ticker} not found.")
        return None, None


Found tickers: ['AAPL', 'DAX', 'DJI', 'MSFT', 'NDX', 'NKX', 'NVDA', 'SPX']


In [9]:
# Create Interactive Plots for each Ticker
for ticker in tickers:
    df_prices, df_equity = load_ticker_data(ticker)
    
    if df_prices is None or df_equity is None:
        continue
        
    print(f"Generating dashboard for {ticker}...")

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(f'{ticker} Price Action', 'Strategy Equity Curves'),
        row_heights=[0.4, 0.6]
    )

    # === 1. Price Chart (Candlestick) ===
    # We have Open and Close. We'll use them for High/Low as well since we don't have true High/Low.
    # This creates a "red/green bar chart" effect.
    
    if 'Open' in df_prices.columns and 'Close' in df_prices.columns:
        # Approximate High/Low
        high = df_prices[['Open', 'Close']].max(axis=1)
        low = df_prices[['Open', 'Close']].min(axis=1)
        
        fig.add_trace(go.Candlestick(
            x=df_prices.index,
            open=df_prices['Open'],
            high=high,
            low=low,
            close=df_prices['Close'],
            name=f'{ticker} Price'
        ), row=1, col=1)
    else:
        # Fallback to Line if Open is missing
        fig.add_trace(go.Scatter(
            x=df_prices.index,
            y=df_prices['Close'],
            mode='lines',
            name=f'{ticker} Close',
            line=dict(color='black', width=1)
        ), row=1, col=1)
    
    # === 2. Equity Curves ===
    colors = {
        'Buy & Hold': 'gray',
        'Base RF': 'red',
        'Sent RF': 'orange',
        'Base LSTM': 'blue',
        'Sent LSTM': 'green'
    }

    for col in df_equity.columns:
        fig.add_trace(go.Scatter(
            x=df_equity.index,
            y=df_equity[col],
            mode='lines',
            name=col,
            line=dict(width=2, color=colors.get(col, 'purple'))
        ), row=2, col=1)

    # === Layout Settings ===
    fig.update_layout(
        title=f'Market Analysis Dashboard: {ticker}',
        yaxis_title='Price ($)',
        yaxis2_title='Equity ($)',
        xaxis_rangeslider_visible=False,
        height=800,
        template='plotly_white',
        hovermode='x unified',
        dragmode='zoom'
    )

    # Add Range Slider to the bottom axis
    fig.update_xaxes(
        rangeslider_visible=True,
        row=2, col=1
    )
    
    fig.show()


Generating dashboard for AAPL...


Generating dashboard for DAX...


Generating dashboard for DJI...


Generating dashboard for MSFT...


Generating dashboard for NDX...


Generating dashboard for NKX...


Generating dashboard for NVDA...


Generating dashboard for SPX...


In [17]:
# === Comprehensive Summary ===
metrics_path = os.path.join(DATA_DIR, 'model_metrics.csv')
if os.path.exists(metrics_path):
    print("\n=== Comprehensive Results Summary ===")
    df_metrics = pd.read_csv(metrics_path)
    
    pivot_return = df_metrics.pivot(index='Ticker', columns='Model', values='Return')
    print("\nCumulative Return (%):")
    display(pivot_return.style.background_gradient(cmap='RdYlGn', axis=None, vmin=-100, vmax=100).format("{:.2f}%").set_properties(**{'width': '100px'}))
    
    pivot_acc = df_metrics.pivot(index='Ticker', columns='Model', values='Accuracy').drop(columns=['Buy & Hold'], errors='ignore')
    print("\nModel Accuracy (%):")
    display(pivot_acc.style.background_gradient(cmap='Blues', axis=None, vmin=0, vmax=1).format("{:.2%}", na_rep="-").set_properties(**{'width': '100px'}))
    
else:
    print("Metrics file not found.")



=== Comprehensive Results Summary ===

Cumulative Return (%):


Model,Base LSTM,Base RF,Buy & Hold,Sent LSTM,Sent RF
Ticker,,,,,
AAPL,59.92%,4.99%,50.63%,14.28%,16.67%
DAX,9.93%,-19.92%,44.47%,28.57%,-25.15%
DJI,35.84%,18.20%,40.17%,17.49%,6.99%
MSFT,6.13%,-74.68%,40.04%,12.04%,-57.09%
NDX,32.28%,13.79%,69.95%,50.12%,23.09%
NKX,12.32%,-47.47%,48.59%,21.13%,-37.07%
NVDA,56.52%,-56.45%,246.12%,112.73%,-52.10%
SPX,9.79%,-22.69%,57.79%,58.02%,-1.47%



Model Accuracy (%):


Model,Base LSTM,Base RF,Sent LSTM,Sent RF
Ticker,,,,
AAPL,48.23%,50.52%,45.74%,50.52%
DAX,50.71%,51.92%,50.18%,51.22%
DJI,52.84%,51.39%,49.65%,49.65%
MSFT,49.47%,52.09%,49.29%,51.74%
NDX,50.35%,50.70%,50.00%,50.52%
NKX,51.42%,54.01%,51.77%,51.92%
NVDA,50.18%,52.09%,46.63%,50.87%
SPX,49.65%,52.26%,50.53%,46.69%


In [11]:
# # Export to HTML
# output_path = os.path.join(RESULTS_DIR, 'interactive_presentation.html')
# fig.write_html(output_path)
# print(f"Interactive presentation saved to: {output_path}")